In [2]:
from langchain_groq import ChatGroq

In [ ]:
llm = ChatGroq(
    temperature=0,
    #groq_api_key = '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',
    model_name="llama-3.3-70b-versatile",
)
response = llm.invoke("What composed the Mona Lisa?")
print(response.content)

The Mona Lisa, a famous painting by Leonardo da Vinci, is composed of several key elements:

1. **Support**: The painting is done on a single piece of poplar wood, which was a common support material for paintings during the Renaissance.
2. **Ground**: The wood panel was prepared with a layer of gesso, a mixture of water, glue, and whiting (calcium carbonate), to create a smooth surface for painting.
3. **Paint**: The painting is executed in oil paint, with a range of pigments including:
	* Lead white (for highlights and skin tones)
	* Ultramarine blue (for the sky and clothing)
	* Vermilion (for the subject's lips and clothing)
	* Earth oxides (for the subject's skin and hair)
	* Carbon black (for shading and outlines)
4. **Techniques**: Da Vinci employed several techniques to achieve the painting's distinctive effects, including:
	* Sfumato: a method of blending colors to create a soft, hazy effect
	* Chiaroscuro: the use of strong contrasts between light and dark to create a sense o

In [17]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://careers.nike.com/maintenance-manager-elc-day-shift/job/R-62423")
page_data = loader.load().pop().page_content
print(page_data)






















Maintenance Manager, ELC (day shift)










































Skip to main content
Open Virtual Assistant










Home


Career Areas


Total Rewards


Life@Nike


Purpose










Language





Select a Language

  Deutsch  
  English  
  Español (España)  
  Español (América Latina)  
  Français  
  Italiano  
  Nederlands  
  Polski  
  Tiếng Việt  
  Türkçe  
  简体中文  
  繁體中文  
  עִברִית  
  한국어  
  日本語  








Careers


















Close Menu







Careers






Chat






                                Home
                            



                                Career Areas
                            



                                Total Rewards
                            



                                Life@Nike
                            



                                Purpose
                            










Jordan Careers







Converse Careers










Language











Menu



Return to Previo

In [9]:
pip install beautifulsoup4

  Using cached beautifulsoup4-4.13.4-py3-none-any.whl.metadata (3.8 kB)
Using cached beautifulsoup4-4.13.4-py3-none-any.whl (187 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [18]:
from langchain_core.prompts import PromptTemplate
prompt_extract = PromptTemplate.from_template(
    """
    ### SCRAPED TEXT FROM WEB PAGE:
    {page_data}
    ### TASK:
    The scraped text above is from a job posting. Extract the following information in JSON format containing the keys:
    'role', 'experience', 'skills', 'description'.
    Only return the valid JSOn.kwargs=## Valid JSON (NO PREAMBLE):
    """
    )

chain_extract = prompt_extract | llm
res = chain_extract.invoke(input={"page_data": page_data})
print(res.content)


```json
{
    "role": "Maintenance Manager, ELC (day shift)",
    "experience": "Proven professional expertise in a maintenance/engineering environment within a multinational company",
    "skills": [
        "theoretical technical background, relevant electromechanical engineering degree or equivalent through experience",
        "expertise in all aspects of automation: mechanics, electrics, pneumatics, PLC",
        "experience with Material Handling Equipment",
        "positive, open-minded and solution-oriented approach",
        "excellent communication skills in Dutch and English"
    ],
    "description": "Lead, mentor, drive and inspire a team of maintenance technicians and associates to improve availability and reliability of technical assets at the European Logistics Campus"
}
```


In [ ]:
import pandas as pd

df = pd.DataFrame([res.content])

In [ ]:
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.create_collection(name='job-postings')

if not collection.count():
    collection.add(
        documents=df['description'].tolist(),
        metadatas=df.to_dict(orient='records'),
        ids=[str(i) for i in range(len(df))]
    )